In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import joblib

file_path = r"C:\Users\ASUS\OneDrive\Documents\jupyter\UAS\tatoeba_data.csv"

# Muat data Tatoeba
data = pd.read_csv(file_path, sep='\t')


# Muat dataset dengan header yang sesuai
data = pd.read_csv(file_path, delimiter='\t', header=None, names=['ID', 'Language Code', 'Source Sentence'])

# Tampilkan beberapa baris pertama untuk memahami struktur data
print(data.head())

   ID Language Code         Source Sentence
0   1           cmn                  我們試試看！
1   2           cmn                 我该去睡觉了。
2   3           cmn                 你在干什麼啊？
3   4           cmn                  這是什麼啊？
4   5           cmn  今天是６月１８号，也是Muiriel的生日！


In [18]:
# Hapus baris dengan nilai NaN secara eksplisit
data = data.dropna()

# Sampling data untuk mengurangi ukuran
data_sample = data.sample(frac=0.1, random_state=42)  # Menggunakan 10% dari data

# Ekstrak fitur dan target
X = data_sample['Source Sentence']
y = data_sample['Language Code']

# Praproses teks dengan membatasi jumlah fitur
vectorizer = TfidfVectorizer(max_features=10000)  # Mengatur jumlah fitur maksimum
X_vectorized = vectorizer.fit_transform(X)

# Bagi data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

Training data shape: (969517, 10000)
Test data shape: (242380, 10000)


In [19]:
# Sel 3: Train the Model
from sklearn.linear_model import SGDClassifier

# Inisialisasi dan latih model
model = SGDClassifier(loss='log_loss', max_iter=10, tol=1e-3)
model.fit(X_train, y_train)

print("Model trained successfully!")

In [ ]:
y_pred = model.predict(X_test)

# Evaluasi
print(classification_report(y_test, y_pred))

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

          \N       0.00      0.00      0.00         8
         acm       0.00      0.00      0.00         1
         afb       0.00      0.00      0.00         3
         afr       0.00      0.00      0.00        86
         ain       0.00      0.00      0.00         2
         akl       0.00      0.00      0.00         2
         alt       0.00      0.00      0.00        37
         amh       0.00      0.00      0.00         4
         ang       0.00      0.00      0.00         9
         aoz       0.00      0.00      0.00         1
         apc       0.00      0.00      0.00         2
         ara       0.95      0.32      0.48      1251
         arg       0.00      0.00      0.00         3
         arn       0.00      0.00      0.00        14
         arq       0.00      0.00      0.00        40
         arz       0.00      0.00      0.00        20
         asm       0.00      0.00      0.00       198
         ast       0.00    

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
joblib.dump(model, 'language_prediction_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

print("Model and vectorizer saved successfully!")

Model and vectorizer saved successfully!


In [ ]:
model = joblib.load('language_prediction_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')

# Fungsi untuk memprediksi bahasa
def predict_language(sentence):
    sentence_vectorized = vectorizer.transform([sentence])
    return model.predict(sentence_vectorized)[0]

# Contoh penggunaan model
test_sentence = "This is a sample sentence."
language = predict_language(test_sentence)
print(f"The predicted language for the sentence '{test_sentence}' is: {language}")

The predicted language for the sentence 'This is a sample sentence.' is: eng
